<a href="https://colab.research.google.com/github/alokchoudharyguliya/FineTuning/blob/main/distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch datasets transformers

: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMasekdLM, Trainer, TrainingArguments


In [14]:
from datasets import load_dataset

In [12]:
STUDENT_MODEL="google-bert/bert-base-uncased"
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(STUDENT_MODEL)
model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### DATASET [SST2](https://huggingface.co/datasets/stanfordnlp/sst2)

In [15]:
dataset=load_dataset("sst2",split="train[:10]")
dataset


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 10
})

In [16]:
dataset[0]

{'idx': 0,
 'sentence': 'hide new secretions from the parental units ',
 'label': 0}

In [17]:
!pip install langchain_ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [26]:
from langchain_ollama import ChatOllama
llm_engine=ChatOllama(
    model="llama3",
    base_url="127.0.0.1:11434",
    temperature=0.3
)

In [27]:
test_prompt="I am so happy today"
response=llm_engine.invoke(test_prompt)
print(response.content)

ConnectError: [Errno 111] Connection refused

We want the teacher to give reasoning to the student that is why a particular label for the sentence/phrase, so adding one column for the same in the initial dataset

In [ ]:
def generate_rationale(input_text):
  """
  Uses Ollama's deepseek R1 model to generate a step-by-step rationale for the given input
  """
  prompt=f"Explain step-by-step reasoning before answering : {input_text}"# Chain of Thought prompting required for step by step distillation
  response=llm_engine.invoke(prompt)
  return response.content if hasattr(response, "content") else response
print(generate_rationale(test_prompt))

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(STUDENT_MODEL)

In [ ]:
# prepare dataset with rationales
def process_data(example):
  input_text=example["sentences"]
  rationale=generate_rationale(input_text)
  label=example["label"]
  # tokenize input and rationale
  input_enc=tokenizer(input_text, truncation=True, padding="max_length",max_length=256)
  rationale_enc=tokenizer(rationale, truncation=True, padding="max_length", max_length=256)
  return {
      'input_ids':input_enc['input_ids'],
      'attention_mask':input_enc['attention_mask'],
      'labels':label,
      'rationale_ids':rationale_enc['input_ids'],
      'rationale_mask':rationale_enc['attention_mask']
  }
# apply function to dataset
processed_dataset=dataset.map(process_data)

In [ ]:
processed_dataset

In [2]:
!pip install -U accelerate==0.26.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.1
    Uninstalling accelerate-1.10.1:
      Successfully uninstalled accelerate-1.10.1


Accelerate is a library that enables the same PyTorch code to be run across any distributed configuration by adding just four lines of code!

In [11]:
import accelerate
accelerate.__version__

'0.26.0'

In [ ]:
training_args=TrainingArguments(
    output_dir="./results", # Directory to save the model and checkpoints
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=False
)

In [ ]:
# Custom loss function for multi-task learning:
# - Combine standarad classification loss(label_loss)
# - Computes rationale loss using rationale token IDs
# - Combine both losses (weighted: rationale loss contributes 0.5x)
class MultiTaskTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
    labels=inputs.pop("labels")
    rationale_ids=inputs.pop("rationale_ids",None)
    outputs=model(**inputs)

    loss_fn=torch.nn.CrossEntropyLoss()
    label_loss=loss_fn(outputs.logits, labels)
    if rationale_ids is not None:
      rationale_outputs=model(input_ids=rationale_ids, attention_mask=inputs['attention_mask'])
      rationale_loss=loss_fn(rationale_)
